<a href="https://colab.research.google.com/github/123ranika/Research-paper/blob/main/Transformer_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [ ]:
# Define the models we'll use in our ensemble
model_names = [
    "sagorsarker/bangla-bert-base",
    "bert-base-multilingual-uncased",
    "distilbert/distilbert-base-uncased",
    "FacebookAI/xlm-roberta-base"
]

In [10]:
# Load and preprocess the datasets
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    # Ensure 'text' column exists and contains string data
    if 'text' not in df.columns:
        raise ValueError(f"'text' column not found in {file_path}")
    df['text'] = df['text'].astype(str)
    return df

try:
    train_df = pd.read_csv('/content/Traning.xlsx - Sheet1 (7).csv')
    val_df = pd.read_csv('/content/Val.xlsx - Sheet1 (2).csv')
    test_df = pd.read_csv('/content/Test.xlsx - Sheet1 (2).csv')
    temp_df = pd.read_csv('/content/Val.xlsx - Sheet1 (2).csv')
    val_df['labels'] = temp_df['labels']
except Exception as e:
    print(f"Error loading data: {e}")
    raise

In [11]:
# Combine train and validation for stratified split
combined_df = pd.concat([train_df, val_df], ignore_index=True)

In [13]:
# Perform stratified split
train_df, val_df = train_test_split(combined_df, test_size=0.1, stratify=combined_df['labels'], random_state=42)

In [14]:
# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [15]:
# Helper function for computing metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [16]:
# Function to train a single model
def train_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_function(examples):
        # Ensure 'text' is a list of strings
        texts = [str(text) for text in examples['text']]
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_val = val_dataset.map(tokenize_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.split('/')[-1]}",
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs_{model_name.split("/")[-1]}',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    return trainer, tokenizer


In [17]:
# Train all models
trained_models = []
for model_name in model_names:
    print(f"Training model: {model_name}")
    try:
        trainer, tokenizer = train_model(model_name)
        trained_models.append((trainer, tokenizer))
    except Exception as e:
        print(f"Error training model {model_name}: {e}")
        continue


Training model: sagorsarker/bangla-bert-base


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-ccd5c68c9106>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Error training model FacebookAI/xlm-roberta-base: 


In [18]:
# Function to get predictions from a single model
def get_predictions(trainer, tokenizer, dataset):
    def tokenize_function(examples):
        # Ensure 'text' is a list of strings
        texts = [str(text) for text in examples['text']]
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    predictions = trainer.predict(tokenized_dataset)
    return predictions.predictions

In [19]:
# Get predictions from all models
all_predictions = []
for trainer, tokenizer in trained_models:
    print(f"Getting predictions from model: {trainer.model.name_or_path}")
    try:
        predictions = get_predictions(trainer, tokenizer, test_dataset)
        all_predictions.append(predictions)
    except Exception as e:
        print(f"Error getting predictions from model {trainer.model.name_or_path}: {e}")
        continue


In [20]:
# Ensemble predictions using majority voting
if all_predictions:
    ensemble_predictions = np.stack(all_predictions)

    # Apply majority voting along the model axis for each sample
    majority_votes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), 0, ensemble_predictions.argmax(axis=2))

    # Create a DataFrame with predictions
    results_df = pd.DataFrame({
        'text': test_df['text'],
        'label': majority_votes
    })

    # Save the results to a new CSV file
    #results_df.to_csv('ensemble_test_results.csv', index=False)
    print("Ensemble predictions have been saved to 'ensemble_test_results.csv'")
else:
    print("No predictions were made. Check the errors above.")


No predictions were made. Check the errors above.


In [ ]:


# Rest of the code remains the same...





